## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import pprint

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Waingapu,ID,-9.6567,120.2641,77.45,74,61,12.35,broken clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,82.80,74,100,10.78,overcast clouds
2,2,Barrow,US,71.2906,-156.7887,28.42,86,100,4.61,mist
3,3,The Valley,AI,18.2170,-63.0578,82.26,78,21,19.01,few clouds
4,4,Yellowknife,CA,62.4560,-114.3525,51.39,73,99,10.96,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Waingapu,ID,-9.6567,120.2641,77.45,74,61,12.35,broken clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,82.80,74,100,10.78,overcast clouds
3,3,The Valley,AI,18.2170,-63.0578,82.26,78,21,19.01,few clouds
5,5,Grand Gaube,MU,-20.0064,57.6608,71.38,88,91,10.00,moderate rain
8,8,Hobyo,SO,5.3505,48.5268,84.63,65,99,26.73,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.isna().sum())
isna_countries = preferred_cities_df[preferred_cities_df["Country"].isnull()]
isna_countries

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
75,75,Khorixas,NaN,-20.3667,14.9667,79.23,23,0,19.60,clear sky
325,325,Windhoek,NaN,-22.5594,17.0832,70.29,32,0,13.80,clear sky
496,496,Eenhana,NaN,-17.4667,16.3333,79.43,22,0,7.49,clear sky
515,515,Opuwo,NaN,-18.0607,13.8400,78.03,24,0,12.35,clear sky


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
# clean_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Waingapu,ID,77.45,broken clouds,-9.6567,120.2641,
1,Hithadhoo,MV,82.80,overcast clouds,-0.6000,73.0833,
3,The Valley,AI,82.26,few clouds,18.2170,-63.0578,
5,Grand Gaube,MU,71.38,moderate rain,-20.0064,57.6608,
8,Hobyo,SO,84.63,overcast clouds,5.3505,48.5268,
9,Dingle,PH,80.06,broken clouds,10.9995,122.6711,
11,Rikitea,PF,73.02,scattered clouds,-23.1203,-134.9692,
12,Naifaru,MV,83.59,light rain,5.4444,73.3657,
14,Butaritari,KI,81.84,few clouds,3.0707,172.7902,
16,Victoria,HK,85.32,overcast clouds,22.2855,114.1577,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# check Null
print(hotel_df.isnull().values.any())

False


In [10]:
x_df = hotel_df.loc[hotel_df["Hotel Name"] == ""]

# number rows with "" from hotelname
print(x_df["Hotel Name"].count())

# display data frame with "" as Hotel name
x_df.head()

26


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
42,Makali,SL,82.81,light rain,8.6333,-11.6667,
124,Kayerkan,RU,73.53,overcast clouds,69.3497,87.7602,
168,Saskylakh,RU,71.01,broken clouds,71.9167,114.0833,
192,Jamame,SO,84.09,light rain,0.0722,42.7506,
199,Yaan,NG,85.41,broken clouds,7.3833,8.5667,


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df.loc[hotel_df["Hotel Name"] == ""].index)
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Waingapu,ID,77.45,broken clouds,-9.6567,120.2641,Pondok Wisata Elim
1,Hithadhoo,MV,82.80,overcast clouds,-0.6000,73.0833,Scoop Guest House
3,The Valley,AI,82.26,few clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort
5,Grand Gaube,MU,71.38,moderate rain,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
8,Hobyo,SO,84.63,overcast clouds,5.3505,48.5268,Iftin hotel hobyo


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))